In [25]:
import tensorflow as tf

# The file path to save the data
save_file = './model.ckpt'

# Two Tensor Variables: weights and bias
weights = tf.Variable(tf.truncated_normal([2, 3]))
bias = tf.Variable(tf.truncated_normal([3]))

# Class used to save and/or restore Tensor Variables
saver = tf.train.Saver()

with tf.Session() as sess:
    # Initialize all the Variables
    sess.run(tf.global_variables_initializer())

    # Show the values of weights and bias
    print('Weights:')
    print(sess.run(weights))
    print('Bias:')
    print(sess.run(bias))

    # Save the model
    saver.save(sess, save_file)

Weights:
[[ 1.81999266 -0.14006907  1.34111786]
 [ 0.26232857 -1.18705428 -0.52387816]]
Bias:
[-0.52481389 -0.35811204  0.25916123]


In [26]:
# Remove the previous weights and bias
tf.reset_default_graph()

# Two Variables: weights and bias
weights = tf.Variable(tf.truncated_normal([2, 3]))
bias = tf.Variable(tf.truncated_normal([3]))

# Class used to save and/or restore Tensor Variables
saver = tf.train.Saver()

with tf.Session() as sess:
    # Load the weights and bias
    saver.restore(sess, save_file)

    # Show the values of weights and bias
    print('Weight:')
    print(sess.run(weights))
    print('Bias:')
    print(sess.run(bias))

Weight:
[[ 0.93899167  0.54777712  0.48947647]
 [ 0.19502904  0.97432697  1.81425142]]
Bias:
[-0.59108222 -0.10917336 -0.43077204]


### Save a trained model

In [46]:
tf.reset_default_graph()

from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

learning_rate = 0.001
n_input = 28*28
n_classes = 10

mnist = input_data.read_data_sets(".", one_hot=True)

# features and labels
features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_classes])

# weights and bias
weights = tf.Variable(tf.random_normal([n_input, n_classes]))
bias = tf.Variable(tf.random_normal([n_classes]))

# logits
logits = tf.add(tf.matmul(features, weights), bias)

# loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# calculate accuracy 
correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


In [47]:
# train the model and save the weights
import math

save_file = './train_model.ckpt'
batch_size = 128
n_epochs = 100

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(n_epochs):
        total_batch = math.ceil(mnist.train.num_examples/batch_size)
        # loop over all batches
        for i in range(total_batch):
            batch_features, batch_labels = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={features: batch_features, labels: batch_labels})
            
        # print status for every 10 epochs
        if epoch % 10 == 0:
            valid_accuracy = sess.run(accuracy, 
                                      feed_dict={features: mnist.validation.images, 
                                                 labels: mnist.validation.labels})
            print("Epoch {:<3} - Validation Accuracy: {}".format(epoch, valid_accuracy))
            
    # save the model
    saver.save(sess, save_path=save_file)
    print('Trained Model Saved.')

Epoch 0   - Validation Accuracy: 0.11999999731779099
Epoch 10  - Validation Accuracy: 0.2619999945163727
Epoch 20  - Validation Accuracy: 0.4002000093460083
Epoch 30  - Validation Accuracy: 0.48739999532699585
Epoch 40  - Validation Accuracy: 0.5511999726295471
Epoch 50  - Validation Accuracy: 0.6019999980926514
Epoch 60  - Validation Accuracy: 0.6385999917984009
Epoch 70  - Validation Accuracy: 0.6682000160217285
Epoch 80  - Validation Accuracy: 0.6917999982833862
Epoch 90  - Validation Accuracy: 0.7081999778747559
Trained Model Saved.


### Load a trained model

In [48]:
saver = tf.train.Saver()

# Launch the graph
with tf.Session() as sess:
    saver.restore(sess, save_file)

    test_accuracy = sess.run(
        accuracy,
        feed_dict={features: mnist.test.images, labels: mnist.test.labels})

print('Test Accuracy: {}'.format(test_accuracy))

Test Accuracy: 0.7226999998092651
